In [321]:
import sys
sys.path.append('../../bluesky')  # Adjust the path as necessary

# Now you can import bluesky modules

In [322]:
import bluesky
from bluesky.network.client import Client
import os


from bluesky.network.client import Client


import google.generativeai as genai


from dotenv import load_dotenv, find_dotenv


import sys


from contextlib import contextmanager


from io import StringIO


from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [323]:
# Load the .env file
load_dotenv(find_dotenv())

True

In [324]:
# capture output information from the bluesky client and return it as a string
@contextmanager
def capture_stdout():
    new_stdout = StringIO()
    old_stdout = sys.stdout
    sys.stdout = new_stdout
    try:
        yield new_stdout
    finally:
        sys.stdout = old_stdout


def update_until_complete(client):
    complete_output = ""
    while True:
        with capture_stdout() as captured:
            client.update()
        new_output = captured.getvalue()
        if not new_output.strip():
            # No more new output, break the loop
            break
        complete_output += new_output
    client.update()
    return complete_output

In [325]:
# connect to client
client = Client()
client.connect("127.0.0.1", 11000, 11001)
client.update()

Client b'\x00\xac\x80\xc0\xe6' connected to host b'\x00\xcd\xb7)%'
Client active node changed.


In [326]:
client.update()

In [327]:
# # example commands
# command = 'CRE BZB994 A320 -15.101549,-14.504519 48 FL150 150'
# command = 'GETACIDS'
# #command = 'PAN BZB994'
# # sending command to the client. can be a plugin command or a pre-defined command
# client.send_event(b'STACK', command)

In [328]:
# sim_output = update_until_complete(client)
# print("All captured output:\n", sim_output)
# print('-------------------')

All captured output:
 
-------------------


In [329]:
client.update()

In [330]:
# initialize the GPT-3.5 model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [331]:

@tool
def get_all_aircraft_info(command: str) -> str:
    """Returns each aircraft information. Use command GETACIDS to get all aircrafts."""
    client.send_event(b'STACK', command)
    sim_output = update_until_complete(client)
    return sim_output


In [332]:
import time
@tool
def send_command_to_simulator(command: str, args: list = None) -> str:

    """Sends a command with optional arguments to the simulator and returns the output.
    Parameters:
    - command (str): The command to send to the simulator.

    - args (list, optional): A list of arguments to include with the command. Defaults to None.

    Returns:

    str: The output from the simulator.
    """
    

    # Ensure args is a list if not provided

    if args is None:

        args = []


    # Convert the command and its arguments into a byte string to be sent

    # Note: Assuming that arguments are separated by spaces in the command string

    command_with_args = ' '.join([command] + args)


    # Send the command to the simulator
    #client.update()
    client.send_event(b'STACK', command_with_args)
    
    # wait 1 second
    time.sleep(1)

    # Wait for and retrieve the output from the simulator

    sim_output = update_until_complete(client)

    return sim_output

In [333]:

tools = [send_command_to_simulator]

In [334]:
# Create a chat prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, that can also use tools to answer questions",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [335]:
llm_with_tools = llm.bind_tools(tools)

In [336]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [337]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [366]:
input1 = "Create an airraft for me. it should be a A320 with the A10 callsign, at -15.101549,-14.504519, at 48 heading, flying with the speed of 150 at flight level150. Here are the command instructions: CRE acid,type,lat,lon,hdg,alt,spd. Altitude can be given as altitude [ft] or flight level [-]. In order to give the altitude as flight level, use the letters FL"

input2 = "Get all aircrafts in the simulator. Only use GETACIDS command with no arguments."

out_lst = list(agent_executor.stream({"input": input2}))



> Entering new AgentExecutor chain...

Invoking: `send_command_to_simulator` with `{'command': 'GETACIDS'}`


BZB991
Aircraft ids: ['BZB999', 'BZB991', 'BZB992', 'BZB993', 'BZB994', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A320FL150', 'A320FL150NEW', 'A320_A9', 'A10']
Info on BZB999 A320 index = 0
Pos: S14'16'57.0"  W013'33'51.0"
Hdg: 048   Trk: 048
Alt: 15000 ft  V/S: 0 fpm
CAS/TAS/GS: 150/188/188 kts   M: 0.300

Info on BZB991 A320 index = 1
Pos: S14'19'10.0"  W013'36'23.0"
Hdg: 048   Trk: 048
Alt: 15000 ft  V/S: 0 fpm
CAS/TAS/GS: 150/188/188 kts   M: 0.300

Info on BZB992 A320 index = 2
Pos: S14'20'34.0"  W013'38'0.0"
Hdg: 048   Trk: 048
Alt: 15000 ft  V/S: 0 fpm
CAS/TAS/GS: 150/188/188 kts   M: 0.300

Info on BZB993 A320 index = 3
Pos: S14'20'59.0"  W013'38'28.0"
Hdg: 048   Trk: 048
Alt: 15000 ft  V/S: 0 fpm
CAS/TAS/GS: 150/188/188 kts   M: 0.300

Info on BZB994 A320 index = 4
Pos: S14'22'29.0"  W013'40'12.0"
Hdg: 048   Trk: 048
Alt: 15000 ft  V/S: 0 fpm
CAS/TAS/GS: 150/188/188 kts  

In [367]:
out_lst

[{'actions': [OpenAIToolAgentAction(tool='send_command_to_simulator', tool_input={'command': 'GETACIDS'}, log="\nInvoking: `send_command_to_simulator` with `{'command': 'GETACIDS'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2pWgmZp5RP1mvXupdzH8AjHB', 'function': {'arguments': '{"command":"GETACIDS"}', 'name': 'send_command_to_simulator'}, 'type': 'function'}]})], tool_call_id='call_2pWgmZp5RP1mvXupdzH8AjHB')],
  'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2pWgmZp5RP1mvXupdzH8AjHB', 'function': {'arguments': '{"command":"GETACIDS"}', 'name': 'send_command_to_simulator'}, 'type': 'function'}]})]},
 {'steps': [AgentStep(action=OpenAIToolAgentAction(tool='send_command_to_simulator', tool_input={'command': 'GETACIDS'}, log="\nInvoking: `send_command_to_simulator` with `{'command': 'GETACIDS'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2pWgmZp5RP1mvXupdzH8AjHB

In [ ]:
# TODO
# 1. connect to a memory block so it will be able to remember the previous commands and responses
# 2. create a vector database with commands and their descriptions (just like voyager did). Input is a prompt, output is top 5 commands
# 3. Make a system that makes the agent write its own commands and test out the commands



In [48]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [77]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, SystemMessage

chat_history = []

In [72]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [111]:
chat_history = []
input1 = "how many aircraft are there in the sky currently?"

result1 = agent_executor.invoke(

    {"input": input1, "chat_history": chat_history})

# chat history should have human message, output result from sim, and output result from openai



> Entering new AgentExecutor chain...

Invoking: `get_all_aircraft_info` with `{'command': 'GETACIDS'}`


KL102
Aircraft ids: ['KL101', 'KL102', 'KL103', 'BBB999']
Info on KL101 B74F index = 0
Pos: N49'04'7.0"  E006'00'2.0"
Hdg: 180   Trk: 180
Alt: 25000 ft  V/S: 0 fpm
CAS/TAS/GS: 340/485/485 kts   M: 0.805

Info on KL102 B74F index = 1
Pos: N49'21'14.0"  E004'21'9.0"
Hdg: 210   Trk: 210
Alt: 25000 ft  V/S: 0 fpm
CAS/TAS/GS: 340/485/485 kts   M: 0.805

Info on KL103 B74F index = 2
Pos: N50'06'37.0"  E003'08'47.0"
Hdg: 239   Trk: 239
Alt: 25000 ft  V/S: 0 fpm
CAS/TAS/GS: 340/485/485 kts   M: 0.805

Info on BBB999 A320 index = 3
Pos: S14'26'56.0"  W013'45'17.0"
Hdg: 048   Trk: 048
Alt: 15000 ft  V/S: 0 fpm
CAS/TAS/GS: 150/188/188 kts   M: 0.300

There are currently 4 aircraft in the sky. The aircraft IDs are: KL101, KL102, KL103, and BBB999.

> Finished chain.


In [112]:
intermediate_steps1 = result1['intermediate_steps'][0][1]

chat_history.extend(
    [

        HumanMessage(content=input1),
        AIMessage(content=result1["output"]),
        AIMessage(content=intermediate_steps1)
    ]

)
# order here matters for some reason. If the order is changed, for the next question the tool will be invoke even though it is not needed

In [113]:
chat_history

[HumanMessage(content='how many aircraft are there in the sky currently?'),
 AIMessage(content='There are currently 4 aircraft in the sky. The aircraft IDs are: KL101, KL102, KL103, and BBB999.'),
 AIMessage(content='KL102\nAircraft ids: [\'KL101\', \'KL102\', \'KL103\', \'BBB999\']\nInfo on KL101 B74F index = 0\nPos: N49\'04\'7.0"  E006\'00\'2.0"\nHdg: 180   Trk: 180\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 340/485/485 kts   M: 0.805\n\nInfo on KL102 B74F index = 1\nPos: N49\'21\'14.0"  E004\'21\'9.0"\nHdg: 210   Trk: 210\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 340/485/485 kts   M: 0.805\n\nInfo on KL103 B74F index = 2\nPos: N50\'06\'37.0"  E003\'08\'47.0"\nHdg: 239   Trk: 239\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 340/485/485 kts   M: 0.805\n\nInfo on BBB999 A320 index = 3\nPos: S14\'26\'56.0"  W013\'45\'17.0"\nHdg: 048   Trk: 048\nAlt: 15000 ft  V/S: 0 fpm\nCAS/TAS/GS: 150/188/188 kts   M: 0.300\n\n')]

In [118]:
input2 = "and which aircraft has the lowest altitude?"
result2 = agent_executor.invoke({"input": input2,
                      "chat_history": chat_history})



> Entering new AgentExecutor chain...


The aircraft with the lowest altitude is BBB999. It is currently at an altitude of 15000 ft.

> Finished chain.


In [121]:
try:
    intermediate_steps2 = result2['intermediate_steps'][0][1]
except IndexError:
    intermediate_steps2 = ''



In [122]:

chat_history.extend(
    [

        HumanMessage(content=input2),
        AIMessage(content=result2["output"]),
        AIMessage(content=intermediate_steps2)
    ]

)
# order here matters for some reason. If the order is changed, for the next question the tool will be invoke even though it is not needed

In [123]:
chat_history

[HumanMessage(content='how many aircraft are there in the sky currently?'),
 AIMessage(content='There are currently 4 aircraft in the sky. The aircraft IDs are: KL101, KL102, KL103, and BBB999.'),
 AIMessage(content='KL102\nAircraft ids: [\'KL101\', \'KL102\', \'KL103\', \'BBB999\']\nInfo on KL101 B74F index = 0\nPos: N49\'04\'7.0"  E006\'00\'2.0"\nHdg: 180   Trk: 180\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 340/485/485 kts   M: 0.805\n\nInfo on KL102 B74F index = 1\nPos: N49\'21\'14.0"  E004\'21\'9.0"\nHdg: 210   Trk: 210\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 340/485/485 kts   M: 0.805\n\nInfo on KL103 B74F index = 2\nPos: N50\'06\'37.0"  E003\'08\'47.0"\nHdg: 239   Trk: 239\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 340/485/485 kts   M: 0.805\n\nInfo on BBB999 A320 index = 3\nPos: S14\'26\'56.0"  W013\'45\'17.0"\nHdg: 048   Trk: 048\nAlt: 15000 ft  V/S: 0 fpm\nCAS/TAS/GS: 150/188/188 kts   M: 0.300\n\n'),
 HumanMessage(content='and which aircraft has the lowest altitude?'),
 AIMessage(

In [125]:
input3 = "What is the aircraft id with the lowest altitude?"
result3 = agent_executor.invoke({"input": input3,
                                 "chat_history": chat_history})



> Entering new AgentExecutor chain...


The aircraft ID with the lowest altitude is BBB999.

> Finished chain.


In [126]:
input4 = "What is the alititude of BBB999?"
result4 = agent_executor.invoke({"input": input4,
                                 "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_all_aircraft_info` with `{'command': 'GETACIDS'}`


The altitude of aircraft BBB999 is 15000 ft.

> Finished chain.


In [128]:
result4['intermediate_steps'][0][1]

''